In [3]:
import cv2
import numpy as np

drawing = False
mask = None

def draw_mask(event, x, y, flags, param):
    global drawing, mask
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            cv2.circle(mask, (x, y), 15, 255, -1)
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False

# Load image
img = cv2.imread("input.jpg")
img_copy = img.copy()

mask = np.zeros(img.shape[:2], dtype=np.uint8)

cv2.namedWindow("Draw Mask")
cv2.setMouseCallback("Draw Mask", draw_mask)

print("Draw over the object with mouse. Press 'i' to inpaint, 'r' to reset, 'q' to quit.")

while True:
    display = img_copy.copy()
    display[mask == 255] = (0, 0, 255)  # show mask in red

    cv2.imshow("Draw Mask", display)
    key = cv2.waitKey(1) & 0xFF

    if key == ord('i'):
        result = cv2.inpaint(img_copy, mask, 3, cv2.INPAINT_TELEA)
        cv2.imshow("Result", result)

    elif key == ord('r'):
        mask[:] = 0

    elif key == ord('q'):
        break

cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0x15e61170) is not the object's thread (0x167f9980).
Cannot move to target thread (0x15e61170)

QObject::moveToThread: Current thread (0x15e61170) is not the object's thread (0x167f9980).
Cannot move to target thread (0x15e61170)

QObject::moveToThread: Current thread (0x15e61170) is not the object's thread (0x167f9980).
Cannot move to target thread (0x15e61170)

QObject::moveToThread: Current thread (0x15e61170) is not the object's thread (0x167f9980).
Cannot move to target thread (0x15e61170)

QObject::moveToThread: Current thread (0x15e61170) is not the object's thread (0x167f9980).
Cannot move to target thread (0x15e61170)

QObject::moveToThread: Current thread (0x15e61170) is not the object's thread (0x167f9980).
Cannot move to target thread (0x15e61170)

QObject::moveToThread: Current thread (0x15e61170) is not the object's thread (0x167f9980).
Cannot move to target thread (0x15e61170)

QObject::moveToThread: Current thread (0x15e61170) is n

Draw over the object with mouse. Press 'i' to inpaint, 'r' to reset, 'q' to quit.


In [1]:
import cv2
import numpy as np
import time

# Start webcam
cap = cv2.VideoCapture(0)
time.sleep(2)

# Capture background
background = None
for i in range(30):
    ret, background = cap.read()

background = np.flip(background, axis=1)  # mirror effect

print("Background captured. Now show the red cloak!")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = np.flip(frame, axis=1)

    # Convert BGR to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Red color range (two ranges in HSV)
    lower_red1 = np.array([0, 120, 70])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 120, 70])
    upper_red2 = np.array([180, 255, 255])

    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)

    mask = mask1 + mask2

    # Remove noise
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))
    mask = cv2.morphologyEx(mask, cv2.MORPH_DILATE, np.ones((3,3), np.uint8))

    # Inverse mask
    mask_inv = cv2.bitwise_not(mask)

    # Extract background where cloak is present
    res1 = cv2.bitwise_and(background, background, mask=mask)

    # Extract foreground where cloak is not present
    res2 = cv2.bitwise_and(frame, frame, mask=mask_inv)

    # Final output
    final_output = cv2.addWeighted(res1, 1, res2, 1, 0)

    cv2.imshow("Invisible Cloak", final_output)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Background captured. Now show the red cloak!


QObject::moveToThread: Current thread (0xbff1990) is not the object's thread (0xbf65af0).
Cannot move to target thread (0xbff1990)

QObject::moveToThread: Current thread (0xbff1990) is not the object's thread (0xbf65af0).
Cannot move to target thread (0xbff1990)

QObject::moveToThread: Current thread (0xbff1990) is not the object's thread (0xbf65af0).
Cannot move to target thread (0xbff1990)

QObject::moveToThread: Current thread (0xbff1990) is not the object's thread (0xbf65af0).
Cannot move to target thread (0xbff1990)

QObject::moveToThread: Current thread (0xbff1990) is not the object's thread (0xbf65af0).
Cannot move to target thread (0xbff1990)

QObject::moveToThread: Current thread (0xbff1990) is not the object's thread (0xbf65af0).
Cannot move to target thread (0xbff1990)

QObject::moveToThread: Current thread (0xbff1990) is not the object's thread (0xbf65af0).
Cannot move to target thread (0xbff1990)

QObject::moveToThread: Current thread (0xbff1990) is not the object's thread